In [5]:
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [1]:
import pandas as pd
# Load the CSV file (replace 'your_file.csv' with the actual file name)
df = pd.read_csv("mogi_tphcm_houses_cleaned_filtered_v2.csv")

In [11]:
df = df[df['Price_USD'] <= 800000]  # Houses ≤ $800K

In [44]:
df = df[df['Distance_km_to_centre'] <= 15]  # Houses ≤ $800K

In [45]:
df.describe()

,Bedrooms,Bathrooms,Area,Price_Numeric,Price_per_m2,Distance_km_to_centre,Population,Price_USD,Price_USD_log
count,60944.000000,60944.000000,60944.000000,6.094400e+04,60944.000000,60944.000000,60944.000000,60944.000000,60944.000000
mean,4.190306,4.002248,77.248556,1.139979e+10,6186.468164,6.062178,433662.607361,445521.872258,12.924283
std,1.730553,1.782961,29.091163,4.417459e+09,2645.506478,2.698670,166472.906842,172641.289752,0.419577
min,1.000000,1.000000,32.000000,4.000000e+09,801.673493,0.000000,147168.000000,156326.331216,11.959707
25%,3.000000,3.000000,56.000000,7.600000e+09,4288.118113,3.834019,310178.000000,297020.029311,12.601558
50%,4.000000,4.000000,72.000000,1.100000e+10,5740.107474,6.289203,474792.000000,429897.410845,12.971304
75%,5.000000,5.000000,90.000000,1.500000e+10,7599.196656,7.148581,490620.000000,586223.742062,13.281459
max,10.000000,10.000000,200.000000,2.045000e+10,24425.989253,14.024251,784173.000000,799218.368344,13.591391


In [38]:
from sklearn.preprocessing import StandardScaler
# Sample Data (replace with your housing data)
X = df[['Bedrooms', 'Bathrooms', 'Area', 'Population', 'Distance_km_to_centre']]
y = df['Price_USD_log']

# Initialize the scaler
scaler = StandardScaler()
# Fit and transform the training data
X = scaler.fit_transform(X)

In [39]:
# Train-Test Split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [100, 500, 700],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]
}

rf = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get best parameters
best_params = grid_search.best_params_
print(best_params)


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 700}


In [40]:
# Initialize and Train Model  
rf_best = RandomForestRegressor(
    max_depth=10, 
    max_features='sqrt', 
    min_samples_leaf=1, 
    min_samples_split=2, 
    n_estimators=700,
    random_state=42
)

rf_best.fit(X_train, y_train)


# Make Predictions  
y_pred = rf_best.predict(X_test)

In [41]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate MSE first
mse = mean_squared_error(y_test, y_pred)  
print(f'Mean Squared Error (MSE): {mse}')

# Now calculate RMSE using MSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')


Mean Squared Error (MSE): 0.08306176542498282
Root Mean Squared Error (RMSE): 0.28820438134244736
Mean Absolute Error (MAE): 0.22694356775239227


In [42]:
print(rf_best.score(X_test,y_test))

0.5353672224421764


In [32]:
rf_best.feature_importances_

array([0.0894595 , 0.04753648, 0.41254939, 0.10138196, 0.34907268])